<a href="https://colab.research.google.com/github/julioscalves/free-code-camp/blob/master/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
TRAIN_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/train-data.tsv"
TEST_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/valid-data.tsv"

train_file_path = tf.keras.utils.get_file("train-data.tsv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("valid-data.tsv", TEST_DATA_URL)

In [ ]:
columns = ['classification', 'message']

train_data = pd.read_csv(train_file_path, sep='\t', names=columns)
test_data = pd.read_csv(test_file_path, sep='\t', names=columns)

mapping = {
    'ham': 0, 'spam': 1
}

train_data['classification'] = train_data['classification'].map(mapping)
test_data['classification'] = test_data['classification'].map(mapping)

train_data_y = train_data.pop('classification')
train_data_y = np.asarray(train_data_y).astype('float32').reshape((-1,1))
test_data_y = test_data.pop('classification')
test_data_y = np.asarray(test_data_y).astype('float32').reshape((-1,1))

train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_data_y))
test_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_data_y))

max_features = 10_000
sequence_length = 40

vectorization = tf.keras.layers.experimental.preprocessing.TextVectorization(
    output_mode='int',
    output_sequence_length=sequence_length,
    max_tokens=max_features
)

train_text = train_dataset.map(lambda x, y: x)
vectorization.adapt(train_text)

In [ ]:
embedding_dim = 8

model = keras.Sequential([
    vectorization,
    keras.layers.Embedding(max_features + 1, embedding_dim),
    keras.layers.Dropout(0.2),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1),
])

loss_function = tf.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy(threshold=0.0)
model.compile(loss=loss_function, optimizer='adam', metrics=metrics)

epochs = 10

history = model.fit(train_dataset, epochs=epochs)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    prediction = model.predict([pred_text])[0][0]
    
    if round(prediction) < 0:
        output = "ham"
    else:
        output = "spam"

    return [prediction, output]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
